## Read in data from IRIS and preview the data

In [ ]:
%pylab inline
#let's ignore all the warnings
import warnings
warnings.filterwarnings('ignore')

from obspy.fdsn import Client
from obspy import UTCDateTime
from matplotlib.dates import date2num
import scipy
from obspy import signal


In [ ]:
signal??

In [ ]:
t1 = UTCDateTime("2010-02-27T06:40:00.000")
t2 = UTCDateTime("2010-02-27T08:30:00.000")

client = Client("IRIS")

#st_raw = client.get_waveforms("IU", "ANMO", "00", "BHZ", t1, t2)
#cat = client.get_events(starttime=t1, endtime=t2, minmagnitude=3)

#get the waveform and station infor from IRIS
st_raw = client.get_waveforms("IU", "ANMO", "00", "BH?", t1, t2)
cat = client.get_events(starttime=t1, endtime=t2, minmagnitude=3)
# The FDSN webservices return StationXML metadata.
inv = client.get_stations(starttime=t1, endtime=t2, network="IU",
      station="ANMO", location="00", channel="BH?", level="response")

In [ ]:
st_raw.plot()

In [ ]:
print inv.get_response?

In [ ]:
# Instrument correction directly from StationXML without the need for temporary
# files.
st_raw.attach_response(inv)
st_raw.remove_response(output="VEL")

In [ ]:
st_raw.plot()
ttt = st_raw[0]
ttt.stats

In [ ]:
#plot the single waveform in a standard way
st_raw[0].stats['processing']

In [ ]:
#plot the single waveform as a day plot
st_raw[0].plot(type='dayplot')

In [ ]:
#plot the single waveform as a day plot
st_raw[0].plot??

In [ ]:
st_raw[0].plot

##Remove mean and trend and filter the data
- Note here that the remove mean, remove trend are applied directly on the data, if you want keep the raw data untouched, please copy it first

In [ ]:
#get a copy of the waveforms, so that we can keep hte raw waveform untouched
st = st_raw.copy()

#remove the mean and the trend
st.detrend('demean').detrend('linear')

#get each component 
tr_BH1 = st[0]
tr_BH2 = st[1]
tr_BHZ = st[2]

In [ ]:
#use bandpass filter to filter the signal, but one with zerophase, the other without zerophase
#param zerophase: If True, apply filter once forwards and once backwards.
#This results in twice the number of corners but zero phase shift in the resulting filtered trace.
tr_BHZ_2 = tr_BHZ.copy()
tr_BHZ.filter('bandpass', freqmin=0.001, freqmax = 0.01, corners=2, zerophase=True)
tr_BHZ_2.filter('bandpass', freqmin=0.001, freqmax = 0.01, corners=2, zerophase=False)
tr_BHZ.plot()
tr_BHZ_2.plot()

In [ ]:
#plot the above two figures in one plot, like the p2 conmmand in sac
from matplotlib.dates import date2num
from matplotlib.dates import DateFormatter

#get the time stamp of the trace
start = date2num(tr_BHZ.stats.starttime)
end = date2num(tr_BHZ.stats.endtime)
times = np.linspace(start, end, tr_BHZ.stats.npts)


fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot_date(times, tr_BHZ.data, "k-", label = 'with zerophase')
ax.plot_date(times, tr_BHZ_2.data, "r-", label = 'without zerophase')
ax.legend(loc=1)

#format the x axis as time
timeFmt = DateFormatter('%H:%M %p')
ax.xaxis.set_major_formatter(timeFmt)
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
plt.xlabel('Time')
plt.ylabel('Velocity')

##Characteristics of the filter
- Use an **`impulse signal`** to see the response of the filter

In [ ]:
def plotSpectrum(y,Fs,label):
    """
    Function to plot the Single-Sided Amplitude Spectrum of y(t)
    """
    n = len(y) # length of the signal
    k = arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[range(n/2)] # one side frequency range
    
    Y = scipy.fft(y)/n # fft computing and normalization
    Y = Y[range(n/2)]
     
    plt.plot(frq,abs(Y),'r', label = label) # plotting the spectrum
    plt.xlabel('Freq (Hz)')
    plt.ylabel('|Y(freq)|')
    plt.legend()
    

- ###Different corners
    - We will use different corners of the filter and apply them on the impulse signal
    - Also try to change the zerophase parameter and see the difference

In [ ]:
#define the impulse signal
impulse = repeat(0.,50); impulse[25] =1.

Fs = 50.0;  # sampling rate
Ts = 1.0/Fs; # sampling interval
t = arange(0,1,Ts) # time vector

zerophase = False

for i in range(4):
    #filter the impulse signal from 1 to 10 Hz with different corners
    filt = signal.filter.bandpass(data=impulse, df = 50, freqmin=1, freqmax = 10, corners=i+1, zerophase=zerophase )
    
    if zerophase:
        label = 'n ' + str(i+1) + ' P 2'
    else:
        label = 'n ' + str(i+1) + ' P 1'
    
    #plot the time domain and frequency domain signal
    plt.figure(figsize=(16,2.5))
    plt.subplot(1,2,1)
    plt.plot(t,filt, label = label)
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.subplot(1,2,2)
    plotSpectrum(filt,Fs, label)
    plt.show()
    

##Different filter band

- ###Quick view
    - this is a quick plot of the result from different filter bands

In [ ]:
#specify your frequency band here
fqb = [(0.001, 0.005), (0.005, 0.01),(0.01, 0.05), (0.05, 0.1)]

for fq in fqb:
    print fq
    freqmin = fq[0]
    freqmax = fq[1]
    tr_filt = st[2].copy()
    tr_filt.filter('bandpass', freqmin= freqmin, freqmax = freqmax, corners=2, zerophase=True)
    fig = plt.figure(figsize = (8,2.5))
    tr_filt.plot(color = 'red', number_of_ticks = 5, tick_format = '%H:%M %p', fig=fig)
    label = 'freq. band: ' + str(freqmin) + 'Hz ~ ' + str(freqmax) + 'Hz'
    plt.annotate(label, xy=(0.67, 0.72),xycoords = 'figure fraction')
    

- ### Filter band with record section
    - Record section plot of the waveforms with different filter bands

In [ ]:
from obspy.core.stream import read, Stream
from obspy.core.util import gps2DistAzimuth

#get data from obspy example FTP
host = 'http://examples.obspy.org/'
# Files (fmt: SAC)
files = ['TOK.2011.328.21.10.54.OKR01.HHN.inv',
'TOK.2011.328.21.10.54.OKR02.HHN.inv', 'TOK.2011.328.21.10.54.OKR03.HHN.inv',
'TOK.2011.328.21.10.54.OKR04.HHN.inv', 'TOK.2011.328.21.10.54.OKR05.HHN.inv',
'TOK.2011.328.21.10.54.OKR06.HHN.inv', 'TOK.2011.328.21.10.54.OKR07.HHN.inv',
'TOK.2011.328.21.10.54.OKR08.HHN.inv', 'TOK.2011.328.21.10.54.OKR09.HHN.inv',
'TOK.2011.328.21.10.54.OKR10.HHN.inv']
# Earthquakes' epicenter
eq_lat = 35.565
eq_lon = -96.792

# Reading the waveforms
st_tok = Stream()
for waveform in files:
	st_tok += read(host + waveform)

# Calculating distance from SAC headers lat/lon
# (trace.stats.sac.stla and trace.stats.sac.stlo)
for tr in st_tok:
	tr.stats.distance = gps2DistAzimuth(tr.stats.sac.stla,
									tr.stats.sac.stlo, eq_lat, eq_lon)[0]
	# Setting Network name for plot title
	tr.stats.network = 'TOK'


In [ ]:
#now define the frequency bands
fqb = [(0.5, 1), (1, 5), (5, 10), (10, 15) ]

#plot the record section
for fq in fqb:
    freqmin = fq[0]
    freqmax = fq[1]
    st_tok_filt = st_tok.copy()
    #print st_tok[0].stats
    st_tok_filt.filter('bandpass', freqmin= freqmin, freqmax = freqmax)
    fig = plt.figure(figsize = (8,6))
    st_tok_filt.plot(type='section', plot_dx=20e3, recordlength=100,time_down=True, linewidth=1, grid_linewidth=.25, fig=fig, color = 'red')
    label = 'freq. band: ' + str(freqmin) + 'Hz ~ ' + str(freqmax) + 'Hz'
    plt.annotate(label, xy=(0.67, 0.86),xycoords = 'figure fraction')


## Plot the Spectrogram

In [ ]:
#plot the spectrogram of the first trace
st_tok[0].spectrogram()
st_tok[0].plot()

In [ ]:
# let's trim the trace and re-plot it
t = tr.stats.starttime
st_tok_trim = st_tok[0].copy().trim(t , t + 50)
st_tok_trim.plot()
st_tok_trim.spectrogram(per_lap=0.9, wlen=None, log=True,
                outfile=None, fmt=None, axes=None, dbscale=False,
                mult=8.0, cmap=None, zorder=None, title=None, show=True,
                sphinx=False, clip=[0.0, 1.0])